## Imports

In [2]:
import numpy as np
import open3d as o3d
from scipy.optimize import least_squares
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Ready the point cloud

In [3]:
pcd = o3d.io.read_point_cloud("noise_filter.pcd")
downpcd = pcd.voxel_down_sample(voxel_size=0.005)
points = np.asarray(downpcd.points)

o3d.visualization.draw_geometries([downpcd])

## DBScan

In [4]:
def visualize_clusters(pcd, labels):
    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0  # Assign black to noise (label = -1)
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcd])

# Convert Open3D point cloud to numpy array
points = np.asarray(downpcd.points)

# Perform DBSCAN clustering
eps = 0.02  # This is the maximum distance between two samples for one to be considered as in the neighborhood of the other.
min_samples = 10  # The number of samples (or total weight) in a neighborhood for a point to be considered as a core point.
db = DBSCAN(eps=eps, min_samples=min_samples).fit(points)
labels = db.labels_

# Visualize the clusters
visualize_clusters(downpcd, labels)

In [7]:
db.labels_

array([0, 1, 1, ..., 1, 0, 0])

In [ ]:
# Function to visualize point cloud with Open3D
def visualize_clusters(pcd, labels):
    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0  # Assign black to noise (label = -1)
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcd])

# Function to apply RANSAC to each cluster to find cylinders
def apply_ransac_to_clusters(pcd, labels):
    unique_labels = np.unique(labels)
    pcd_combined = o3d.geometry.PointCloud()

    for label in unique_labels:
        if label == -1:
            continue  # Skip noise points

        # Extract the points belonging to this cluster
        cluster_indices = np.where(labels == label)[0]
        cluster_points = np.asarray(pcd.points)[cluster_indices]

        # Convert to Open3D point cloud
        cluster_pcd = o3d.geometry.PointCloud()
        cluster_pcd.points = o3d.utility.Vector3dVector(cluster_points)

        # Apply RANSAC to fit a cylinder
        try:
            cylinder_model, inliers = cluster_pcd.segment_cylinder(distance_threshold=0.01,
                                                                   ransac_n=3,
                                                                   num_iterations=1000,
                                                                   radius_bounds=[0.01, 0.1])
            if len(inliers) == 0:
                continue  # Skip if no cylinder was found

            inlier_cloud = cluster_pcd.select_by_index(inliers)
            outlier_cloud = cluster_pcd.select_by_index(inliers, invert=True)

            # Color the inliers (the cylinder) and outliers (the remaining points)
            inlier_cloud.paint_uniform_color([1.0, 0, 0])  # Red for cylinders
            outlier_cloud.paint_uniform_color([0, 1.0, 0])  # Green for remaining points

            # Combine the RANSAC results into a single point cloud
            pcd_combined += inlier_cloud
            pcd_combined += outlier_cloud

        except RuntimeError as e:
            print(f"Skipping cluster {label}: {e}")
            continue

    o3d.visualization.draw_geometries([pcd_combined])

# Load your point cloud (replace with your own point cloud)
pcd = o3d.io.read_point_cloud("path_to_your_point_cloud.ply")

# Convert Open3D point cloud to numpy array
points = np.asarray(pcd.points)

# Perform DBSCAN clustering
eps = 0.02  # This is the maximum distance between two samples for one to be considered as in the neighborhood of the other.
min_samples = 10  # The number of samples (or total weight) in a neighborhood for a point to be considered as a core point.
db = DBSCAN(eps=eps, min_samples=min_samples).fit(points)
labels = db.labels_

# Visualize the clusters (optional, before applying RANSAC)
visualize_clusters(pcd, labels)

# Apply RANSAC to the individual clusters to find cylinders
apply_ransac_to_clusters(pcd, labels)